In [196]:
import numpy as np
import pandas as pd

import logging 

In [2]:
b = True 
2 * (not b) 

0

# Storage

In [197]:
# TODO: Name the columns by letters -> Transfrom numpy array t pandas 
class Board(): 

    def __init__(self, board_length, ships):
        self._board_length = board_length 
        self._board_field = np.zeros((board_length, board_length), dtype=bool) 
        for ship in ships:
            col = ship["col"]
            row = ship["row"]
            ship_length = ship["length"] 
            vertical = ship["vertical"]

            ########################### 
            #### Verify game rules ####
            ###########################
            # TODO: Create an own Exception class for each error 

            # Check basic coordinate compatibility 
            try:
                assert type(col) == int
                assert type(row) == int 
                assert col in range(board_length)
                assert row in range(board_length)
            except Exception: 
                raise ValueError("Ship coordinates are not compatible coordinates within the board!") 

            # Check for border adherence 
            try:
                max_extend = (col + ship_length) * vertical + (row + ship_length) * (1 - vertical) - 1
                assert max_extend < board_length
            except Exception:
                raise ValueError("Ship coordinates and length do not confine with the board!")

            # Check for overlapping 
            board_update = np.zeros_like(self._board_field) 
            if vertical:
                board_update[row, col:(col + ship_length)] = 1
            else:
                board_update[row:(row + ship_length), col] = 1 

            try: 
                assert ~np.any(self._board_field & board_update)
            except Exception:
                raise ValueError("Ship overlaps with another ship!")
            
            # Check for boat touching 
            col_lower = max(0, col - 1)
            row_lower = max(0, row - 1)
            if vertical:
                col_upper = min(board_length, col + ship_length + 1)
                row_upper = min(board_length, row + 1) 
            else:
                col_upper = min(board_length, col + 1)
                row_upper = min(board_length, row + ship_length + 1)   
                
            update_borders = np.zeros_like(self._board_field)
            update_borders[row_lower:row_upper, col_lower:col_upper] = 1
            
            try:
                assert ~np.any(self._board_field & update_borders)
            except Exception:
                raise ValueError("Ship touches other ship!")
            
            # Place ship 
            self._board_field = self._board_field + board_update 
        
    def __str__(self): 
        return "Board with ships/n" + str(self._board_field)
    
    @property
    def board_length(self):
        return np.copy(self._board_length) 
    
    # TODO: This should not be accessible/give back the field, except for test purposes 
    @property
    def board_field(self):
        print("This is for Test purposes only!")
        return np.copy(self._board_field) 

    def dimensions(self):
        return self._board_field.shape 
    
    def game_finished(self, board_status):
        return np.sum(self._board_field) == np.sum(self._board_field & board_status)
    
    def hit(self, row, col):
        return self._board_field[row, col] 

## Test board

In [198]:
ships = list() 
ship_1 = {"col": 0, "row": 0, "length": 3, "vertical": True} 
ship_2 = {"col": 2, "row": 1, "length": 2, "vertical": False}
ship_3 = {"col": 2, "row": 3, "length": 4, "vertical": True} 
ship_4 = {"col": 0, "row": 0, "length": 3, "vertical": True} 
ship_5 = {"col": -1, "row": 0, "length": 3, "vertical": True} 
ship_6 = {"col": 0.5, "row": 0, "length": 3, "vertical": True} 

ships.append(ship_1) # Should work 
# ships.append(ship_2) # Raise ValueError (Boat touching)
# ships.append(ship_3) # Raise ValueError (borders)
# ships.append(ship_4) # Raise ValueError (overlapping)
# ships.append(ship_5) # Raise ValueError (coordinates)
# ships.append(ship_6) # Raise ValueError (coordinates)

board = Board(5, ships)


# Gameplay interface 

## Agents

In [199]:
class Agent():
    def __init__(self, board):
        self.board = board 
        self.length = board.board_length
        self._board_status = np.zeros((self.length, self.length), dtype=bool)
        
    def guess(self, row, col):
        self._board_status[row, col] = self.board.hit(row, col) 

    @property 
    def board_status(self):
        return np.copy(self._board_status)

    def next_move(self):
        raise Exception("The function 'next_move()' is not implemented for this type of class Agent!")
    
    

### Naive Agent 

In [200]:
def convert_index(board_length, index):
    row = index // board_length 
    col = index % board_length 
    return (row, col) 

class Naive(Agent):
    def __init__(self, *args):
        super().__init__(*args) 
        self.moves = np.arange(self.board_status.size)
        np.random.shuffle(self.moves)
        self.current_index = 0
    
    def next_move(self):
        index =  self.moves[self.current_index] 
        coordinates = convert_index(self.length, index)
        self.guess(*coordinates) 
        self.current_index += 1 
        return coordinates 

### Bayes Agent

### Reinforcement Agent 

## Game

In [203]:
def game(board_length, ships, Agent="Naive"): 
    board = Board(board_length, ships) 
    if Agent == "Naive": 
        agent = Naive(board) 
    else:
        raise Exception("Unknown Agent type")
    
    moves = list() 
    while ~board.game_finished(agent.board_status): 
        moves.append(agent.next_move())

    rounds = len(moves)
    num_cells = np.prod(board.dimensions()) 
    # num_ships = 
    print(f"Game finished after {rounds} rounds!") 
    print(f"Summary statistics: \n -> {rounds / num_cells}% of all fields were unveiled") 
    # print(f"/n -> {}% of all guesses were a hit.")

    return (rounds, moves) 

## Test gameplay

In [224]:
board_length = 10
ships = list() 
ship_1 = {"col": 0, "row": 0, "length": 3, "vertical": True} 
ship_2 = {"col": 2, "row": 2, "length": 2, "vertical": False}
ship_3 = {"col": 5, "row": 7, "length": 4, "vertical": True} 
ship_4 = {"col": 0, "row": 0, "length": 3, "vertical": True} 
ship_5 = {"col": -1, "row": 0, "length": 3, "vertical": True} 
ship_6 = {"col": 0.5, "row": 0, "length": 3, "vertical": True} 

ships.append(ship_1) # Should work 
ships.append(ship_2) # Raise ValueError (Boat touching)
ships.append(ship_3) # Raise ValueError (borders)
# ships.append(ship_4) # Raise ValueError (overlapping)
# ships.append(ship_5) # Raise ValueError (coordinates)
# ships.append(ship_6) # Raise ValueError (coordinates)

moves = game(board_length, ships) 

# TODO: Benchmark for hitrate 

Game finished after 95 rounds!
Summary statistics: 
 -> 0.95% of all fields were unveiled
